In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.preprocessing import StandardScaler

import torch
import torch_geometric
import numpy as np
import matplotlib.pyplot as plt

from src.models.nn_layers import MLPModel
from src.models.gnn_layers import myGATConv
from src.models.frameworks import GNN_naive_framework
import src.loading as loading
import src.processing as processing

from visualization.analyse_model import plot_errors_labels_comparison

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/julien-rsbrg/venv/venv_semantic_to_liking/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Data


data = loading.load_data()

def nor_function(a,b):
    return (a or b) and not(a and b)

data["NoExp_Exp"] = data.apply(lambda row: nor_function(row["word1_experience"]>50,row["word2_experience"]>50),axis=1)

scaler = StandardScaler()
data.loc[:,["word1_sc_liking","word2_sc_liking","sc_senenceBERT_mpnet_similarity","sc_depressionCont","sc_NoExp_Exp"]] = scaler.fit_transform(data.loc[:,["word1_liking","word2_liking","senenceBERT_mpnet_similarity","depressionCont","NoExp_Exp"]])
data.loc[:,["word1_sc_liking","word2_sc_liking","sc_senenceBERT_mpnet_similarity","sc_depressionCont","sc_NoExp_Exp"]]


== Load Data: start ==
== Load Data: end ==


,word1_sc_liking,word2_sc_liking,sc_senenceBERT_mpnet_similarity,sc_depressionCont,sc_NoExp_Exp
0,-1.172222,-1.152427,1.118413,-0.229448,-0.891164
1,-1.172222,0.605846,-0.021906,-0.229448,-0.891164
2,-1.172222,0.546244,0.867681,-0.229448,1.122128
3,-1.172222,-0.019980,1.244298,-0.229448,-0.891164
4,-1.172222,0.456840,0.193088,-0.229448,-0.891164
...,...,...,...,...,...
198235,0.456083,0.546244,0.040509,-0.708297,1.122128
198236,0.456083,1.201871,-0.668605,-0.708297,1.122128
198237,0.394638,0.546244,-1.800021,-0.708297,1.122128
198238,0.394638,1.201871,-0.165669,-0.708297,1.122128


In [12]:
subdata = data[data["participant"] == 3]

participant_graph,translator_word_to_index = processing.convert_table_to_graph(
    complete_data_table=subdata,
    node_attr_names=["sc_liking","experience"],
    node_label_names=["sc_liking"],
    edge_attr_names=["senenceBERT_mpnet_similarity"],
    return_word_to_index=True)

print("participant_graph:",participant_graph)
print("participant_graph.x:",participant_graph.x)
print("participant_graph.edge_attr:",participant_graph.edge_attr)
print("participant_graph.y:",participant_graph.y)

Test function convert_table_to_graph
Data(x=[60, 1], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], train_mask=[60], val_mask=[60])
validate: True
is undirected: True
has_self_loop: tensor(False)
end Test function convert_table_to_graph
participant_graph: Data(x=[60, 1], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], train_mask=[60], val_mask=[60])
participant_graph.x: tensor([[ 1.2549],
        [ 1.2549],
        [ 1.2549],
        [ 0.7019],
        [-0.3734],
        [ 0.2103],
        [ 0.6097],
        [ 1.2549],
        [ 0.8248],
        [-0.7421],
        [ 0.5175],
        [ 1.2549],
        [-1.8174],
        [ 1.2549],
        [ 0.7326],
        [ 1.0091],
        [-0.8650],
        [ 0.0567],
        [ 1.2549],
        [-0.2813],
        [ 0.4561],
        [-0.2813],
        [-1.0493],
        [ 1.2549],
        [ 0.1489],
        [ 0.2103],
        [-0.3120],
        [ 0.3025],
        [-0.7114],
        [ 1.2549],
        [ 1.2549],
        [ 1.2242],
      

/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/julien-rsbrg/Documents

In [42]:
## Model

#my_module = MLPModel(c_in=1, c_hidden=5, c_out=1,num_layers=2,dp_rate=0.0)

src_mask = torch.Tensor([True,False]).to(torch.bool)
dst_mask = torch.Tensor([True,False]).to(torch.bool)
my_module = myGATConv(
    in_channels=(1,1),
    out_channels=1,
    heads=1,
    negative_slope=0.0,
    add_self_loops=False,
    edge_dim=1,
    dropout=0.0,
    src_mask=src_mask,
    dst_mask=dst_mask)

print(my_module.lin_src)

"""
my_module = torch_geometric.nn.GATConv(
    in_channels=(1,1),
    out_channels=1,
    heads=1,
    negative_slope=0.0,
    add_self_loops=False,
    edge_dim=1,
    dropout=0.0)


my_module = torch_geometric.nn.GCNConv(
    in_channels=1, 
    out_channels=1
    )
"""

# my_module(x=participant_graph.x,edge_index=participant_graph.edge_index,edge_attr=participant_graph.edge_attr)
print(my_module)
print([param for param in my_module.parameters()])

## Training
complete_model = GNN_naive_framework(my_module,device)

if True:
    opt = complete_model.configure_optimizer(lr=0.1)
    scheduler = complete_model.configure_scheduler(opt,0.1,0.1,10)

    history = complete_model.train([participant_graph],10000,1,opt,scheduler,"train_loss",200)

## Visualization
plot_errors_labels_comparison(complete_model,participant_graph,True)

Linear(1, 1, bias=False)
myGATConv((1, 1), 1, heads=1)
[Parameter containing:
tensor([[[-1.3954]]], requires_grad=True), Parameter containing:
tensor([[[1.1988]]], requires_grad=True), Parameter containing:
tensor([[[-0.1450]]], requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True), Parameter containing:
tensor([[0.4751]], requires_grad=True), Parameter containing:
tensor([[0.5936]], requires_grad=True), Parameter containing:
tensor([[0.7999]], requires_grad=True), Parameter containing:
tensor([[-1.0619]], requires_grad=True), Parameter containing:
tensor([[-1.2301]], requires_grad=True)]
== start training ==
torch.Size([60, 1])
torch.Size([60, 1])
epoch: 1/10000,
 train_loss: 0.8051,
 train_mae: 0.7733,
 epoch_time_duration: 0.0043

torch.Size([60, 1])
torch.Size([60, 1])
epoch: 2/10000,
 train_loss: 0.7630,
 train_mae: 0.7502,
 epoch_time_duration: 0.0033

torch.Size([60, 1])
torch.Size([60, 1])
epoch: 3/10000,
 train_loss: 0.7214,
 train_mae: 0.7266,
 epoch_tim

labels= [ 1.2548742   1.2548742   1.2548742   0.70186496 -0.3734308   0.21030119
  0.60969675  1.2548742   0.8247559  -0.74210364  0.51752853  1.2548742
 -1.8173994   1.2548742   0.7325877   1.0090923  -0.8649945   0.05668751
  1.2548742  -0.28126258  0.45608306 -0.28126258 -1.049331    1.2548742
  0.14885572  0.21030119 -0.3119853   0.3024694  -0.7113809   1.2548742
  1.2548742   1.2241515  -0.3734308   1.2548742   1.2548742   1.2548742
 -1.1107764   1.2241515   1.2548742  -0.28126258 -1.8173994   1.2548742
  1.2548742  -0.0354807   1.2548742   0.8247559   0.60969675 -0.21981712
  0.79403317  1.2548742   1.2548742  -0.43487626  0.42536032  0.76331043
  0.85547864  1.2548742   1.2548742   1.2548742   1.2548742   1.2018709 ]
preds= [ 1.2346146   1.2435536   1.2416316   0.7158235  -0.3901049   0.23356138
  0.6359893   1.2372121   0.84782976 -0.768749    0.53821105  1.2362876
 -1.8088084   1.237316    0.74816537  1.0232892  -0.8918963   0.07567862
  1.2404916  -0.27698845  0.48716974 -0.2

In [24]:
torch.nn.init.ones_(complete_model.update_node_module.lin_src.weight)

complete_model.update_node_module.lin_src.weight

Parameter containing:
tensor([[1.]], device='cuda:0', requires_grad=True)

In [29]:
def create_dummy_graph(num_nodes: int,
                       x_dim: int,
                       edge_attr_dim: int,
                       y_dim: int,
                       complete_model: GNN_naive_framework = None, 
                       ratio_known: float = 0.5):
    assert ratio_known > 0 and ratio_known < 1

    edge_index = torch.Tensor(np.meshgrid(np.arange(num_nodes),np.arange(num_nodes))).reshape((2,-1)).to(torch.int64)
    is_self_loop = edge_index.sum(dim=0) < 0.5
    edges_index = edges_index[~is_self_loop]
    num_edges = edges_index.size(1)

    x = torch.normal(mean=np.zeros((num_nodes,x_dim)), std=np.ones((num_nodes,x_dim)))
    
    node_train_mask = torch.ones(num_nodes,dtype=torch.bool)

    if not(complete_model is None):
        p = 1/num_nodes*torch.ones(num_nodes)
        num_known = max(int(ratio_known*num_nodes),1)

        idx = p.multinomial(num_samples=num_known, replacement=False)
        x[idx,-1] = torch.ones(num_known)


    data_graph = torch_geometric.data.Data(
        x = x, 
        edge_index = edge_index,
        edge_attr = torch.normal(mean=np.zeros((num_edges,edge_attr_dim)), std=np.ones((num_edges,edge_attr_dim))), 
        y = x[...,:y_dim], 
        train_mask = node_train_mask, 
        val_mask = ~node_train_mask
        )
    
    

    return data_graph

create_dummy_graph(None,5)

/tmp/ipykernel_12616/2439438702.py:3: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)



tensor([[0., 1., 2., 3., 4., 0., 1., 2., 3., 4., 0., 1., 2., 3., 4., 0., 1., 2.,
         3., 4., 0., 1., 2., 3., 4.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 3., 3., 3.,
         3., 3., 4., 4., 4., 4., 4.]])

In [32]:
a = torch.tensor([1, 2, 3, 4])
p = torch.tensor([0.1, 0.1, 0.1, 0.7])
n = 2
replace = False

idx = p.multinomial(num_samples=n, replacement=replace)
b = a[idx]
b
print(idx)

tensor([1, 3])


In [8]:
torch.Tensor([[0,2,3],[1,2,3]]).sum(dim=0)

tensor([1., 4., 6.])

In [89]:
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objects as go

from typing import Optional, Iterable

def convert_float_to_hsl(l,min_l,max_l,s=100,h=235):
    if max_l < min_l:
        min_l, max_l = max_l, min_l
        print("Careful: min max lighting exchanged")

    if max_l != min_l:
        scaled_l = (l-min_l)/(max_l-min_l)*100
        scaled_l = min(int(scaled_l),90)
    else:
        scaled_l = 90
    
    return f"hsl({h:d},{s:d}%,{scaled_l:d}%)"


def convert_torch_to_networkx_graph(data: torch_geometric.data.Data, edge_attr_id: int = 0, to_undirected:bool|str = False):
    G = to_networkx(data, to_undirected=to_undirected)
    weights = {e_pair:float(data.edge_attr[e,edge_attr_id]) for e,e_pair in enumerate(G.edges())} 
    nx.set_edge_attributes(G, values = weights, name = 'weight')
    return G

def draw_torch_graph(data: torch_geometric.data.Data, 
               node_annotations: Optional[Iterable] = None,
               node_colors: Optional[Iterable] = None,
               node_color_label: Optional[str] = "",
               title: Optional[str] = ""):
    G = convert_torch_to_networkx_graph(data=data)
    draw_networkx_graph(G,
                        node_annotations=node_annotations,
                        node_colors=node_colors,
                        node_color_label=node_color_label,
                        title=title)

def draw_networkx_graph(
        G: nx.Graph, 
        node_annotations: Optional[Iterable] = None,
        node_colors: Optional[Iterable] = None,
        node_color_label: Optional[str] = "",
        title: Optional[str] = ""):
    
    # adapts: https://plotly.com/python/network-graphs/
    pos = nx.spring_layout(G, k=20*1/np.sqrt(G.number_of_nodes()), weight="weight",seed = 50, iterations=500)


    edge_attr = list(nx.get_edge_attributes(G,"weight").values())
    min_edge_attr,max_edge_attr = np.min(edge_attr),np.max(edge_attr)
    
    edge_traces = []
    for edge in G.edges():
        edge_x = []
        edge_y = []
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)

        color = convert_float_to_hsl(float(G.get_edge_data(edge[0],edge[1])["weight"]),min_edge_attr,max_edge_attr)
        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5,color=color),
            hoverinfo='none', #to change
            mode='lines'
        )
        edge_traces.append(edge_trace)
    

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=node_colors,
            size=10,
            colorbar=dict(
                thickness=15,
                title=dict(
                    text=node_color_label, # to change
                    side='right'
                ),
                xanchor='left',
            ),
            line_width=2))
    
    # colors
    node_text = []
    for node in G.nodes():
        # node_adjacencies.append(len(adjacencies[1]))
        node_text.append(str(node_annotations[node])+f":{node_colors[node]:.2f}")

    # node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    # plot
    fig = go.Figure(data=edge_traces+[node_trace],
             layout=go.Layout(
                title=dict(
                    text=title,
                    font=dict(
                        size=16
                    )
                ),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
    fig.show()

participant_id = 3 # should be before when create participant graph
draw_torch_graph(participant_graph,
           node_annotations=list(translator_word_to_index.keys()),
           node_colors = participant_graph.x[:,0].tolist(),
           node_color_label="Liking for activity",
           title=f"Semantic network of activities with liking for participant {participant_id}")

In [21]:
participant_graph.edge_attr[0,0]

tensor(0.3758)

In [105]:
def add_invert_edge_attr(G:nx.Graph,edge_attr_name:str,new_edge_attr_name:str):
    new_G = G.copy()
    edge_attr = nx.get_edge_attributes(new_G,name=edge_attr_name)
    new_edge_attr = {k:1/(v+1e-6) for k,v in edge_attr.items()}
    nx.set_edge_attributes(new_G, values = new_edge_attr, name = new_edge_attr_name)
    return new_G

def add_opposite_edge_attr(G:nx.Graph,edge_attr_name:str,new_edge_attr_name:str):
    new_G = G.copy() 
    edge_attr = nx.get_edge_attributes(new_G,name=edge_attr_name)
    new_edge_attr = {k:-v for k,v in edge_attr.items()}
    nx.set_edge_attributes(G, values = new_edge_attr, name = new_edge_attr_name)
    return new_G

In [78]:
G_participant_graph = convert_torch_to_networkx_graph(participant_graph,to_undirected="upper")

{(0, 1): 0.3758171498775482, (0, 2): 0.24644935131072998, (0, 3): 0.3473719656467438, (0, 4): 0.3900986611843109, (0, 5): 0.2708401679992676, (0, 6): 0.24829773604869843, (0, 7): 0.24048438668251038, (0, 8): 0.4285988211631775, (0, 9): 0.3784288167953491, (0, 10): 0.3638623356819153, (0, 11): 0.35849863290786743, (0, 12): 0.3351920247077942, (0, 13): 0.41887038946151733, (0, 14): 0.4692533016204834, (0, 15): 0.3000445067882538, (0, 16): 0.3002339005470276, (0, 17): 0.35303884744644165, (0, 18): 0.36069953441619873, (0, 19): 0.22602668404579163, (0, 20): 0.4494587779045105, (0, 21): 0.1873822659254074, (0, 22): 0.38017797470092773, (0, 23): 0.2179943323135376, (0, 24): 0.40745607018470764, (0, 25): 0.28195300698280334, (0, 26): 0.5477363467216492, (0, 27): 0.3350009322166443, (0, 28): 0.41057682037353516, (0, 29): 0.37881577014923096, (0, 30): 0.046650804579257965, (0, 31): 0.4701501429080963, (0, 32): 0.27541953325271606, (0, 33): 0.632486879825592, (0, 34): 0.2855656147003174, (0, 35)

In [112]:
def cluster_nodes(G:nx.Graph, opposite_weight:bool = False, upper_quantile_removed:float=.8):
    """
    Cluster nodes with Kruskal's algorithm. 
    The greater the weight the less likely the two nodes are in the same component.
    """
    edge_attr_name = "weight" 
    if opposite_weight:
        new_G = add_opposite_edge_attr(G,"weight","opposite_weight")
        edge_attr_name = "opposite_weight"

    G_MST = nx.minimum_spanning_tree(new_G, weight=edge_attr_name)

    opposite_weight = list(nx.get_edge_attributes(G_MST,name=edge_attr_name).values())
    q = np.quantile(opposite_weight,upper_quantile_removed)
    for edge,edge_opp_weight in nx.get_edge_attributes(G_MST,edge_attr_name).items():
        if edge_opp_weight >= q:
            G_MST.remove_edge(edge[0],edge[1])

    connected_components_index = sorted(nx.connected_components(G_MST), key=len, reverse=True)
    return connected_components_index, G_MST

def draw_clustering(G:nx.Graph,
                    opposite_weight: bool = True,
                    upper_quantile_removed:float=0.8,
                    node_annotations: Optional[Iterable] = None,
                    node_colors: Optional[Iterable] = None,
                    node_color_label: Optional[str] = ""):
    connected_components_index, G_MST = cluster_nodes(G, opposite_weight = opposite_weight, upper_quantile_removed = upper_quantile_removed)

    words = list(translator_word_to_index.keys())
    for nodes_index in connected_components_index:
        new_component_words = []
        for id in nodes_index:
            new_component_words.append(words[id])
        print("Component's words =", new_component_words)

        n_nodes = len(nodes_index)
        if n_nodes > 1: 
            component_subgraph = G_MST.subgraph(nodes_index)
            weights = list(nx.get_edge_attributes(component_subgraph,"weight").values())
            weight_message = f"min weight = {np.min(weights):.3f}, max weights = {np.max(weights):.3f}"

            draw_networkx_graph(
                component_subgraph,
                node_annotations=node_annotations,
                node_colors = node_colors,
                node_color_label = node_color_label,
                title=f"Component with {n_nodes:d} nodes ("+ weight_message +")")

draw_clustering(G_participant_graph,
                upper_quantile_removed=.7,
                node_annotations=list(translator_word_to_index.keys()),
                node_colors = participant_graph.x[:,0].tolist(),
                node_color_label = "Activity liking")

Component's words = ['Autobiography book', 'Brunch', 'Cinema', 'Comedy podcast', 'Concert', 'Current affairs podcast', 'Dinner party', 'Documentary series', 'Fantasy fiction audiobook', 'History book', 'Karaoke', 'Musical', 'Netflix', 'Reading a novel', 'Reality TV', 'RomCom film', 'Sci-fi film', 'Science non-fiction book', 'Stand up comedy night', 'Theater show', 'Trivia night', 'True crime TV series']


Component's words = ['Art gallery', 'Museum', 'Painting', 'Pottery workshop', 'Gardening', 'Historical tour', 'Home decorating']


Component's words = ['Board games', 'Online chess', 'Poker', 'Sports video games', 'Frisbee']


Component's words = ['Book club', 'Clubbing', 'Pub or bar', 'Farmers market', 'Local restaurant']


Component's words = ['Walk in the park', 'Indoor rock climbing', 'Running outside', 'Hiking']


Component's words = ['Meal prepping', 'Baking cookies', 'Bread making']


Component's words = ['Computer programming', 'Creative writing']


Component's words = ['Chat online']
Component's words = ['Clothes shopping']
Component's words = ['DIY']
Component's words = ['Knitting']
Component's words = ['Lift weights']
Component's words = ['News articles']
Component's words = ['Play football']
Component's words = ['Practice an instrument']
Component's words = ['Salsa class']
Component's words = ['Spin class']
Component's words = ['Spotify playlist']
Component's words = ['YouTube videos']
